In [2]:
# all necessary libraries
import pandas as pd
import sys
from pathlib import Path

current_file_dir = Path(r"scripts\machine_learning").resolve().parent
parent_dir = current_file_dir.parent.parent
sys.path.insert(0, str(parent_dir))

In [206]:
df_dist = pd.read_csv('../../data/features/afp_distance_features.csv')
df_tree = pd.read_csv('../../data/features/medoid_tree_features.csv')

# 3. Merge on sequence_id
df = df_dist.merge(df_tree, on=['sequence_id', 'label'])

motif_paths = [
    r"..\..\data\motifs\afgp_motifs.txt",
    r"..\..\data\motifs\type1_motifs.txt",
    r"..\..\data\motifs\type2_motifs.txt",
    r"..\..\data\motifs\type3_motifs.txt",
    r"..\..\data\motifs\type4_motifs.txt"
]

positive_reviewed_sequences_path = r"..\..\data\positives\afp_all_c90.faa"
positive_non_reviewed_sequences_path = r"..\..\data\positives\pblast_filtered\pblast_type4_c90.faa"
positive_type1_path = r"..\..\data\positives\pblast_filtered\pblast_type1_c90.faa"
positive_type2_path = r"..\..\data\positives\pblast_filtered\pblast_type2_c90.faa"
positive_type3_path = r"..\..\data\positives\pblast_filtered\pblast_type3_c90.faa"
positive_type4_path = r"..\..\data\positives\pblast_filtered\pblast_type4_c90.faa"

negative_sequences_path = r"..\..\data\negatives\non_afp_c90.faa"

# mappings for HMM
amino_acids = 'ARNDCQEGHILKMFPSTWYV' # Standard 20 amino acids
aa_to_int = {aa: i for i, aa in enumerate(amino_acids)}
int_to_aa = {i: aa for aa, i in aa_to_int.items()}

if 'N' in aa_to_int: aa_to_int['B'] = aa_to_int['N'] # B -> N
if 'A' in aa_to_int: aa_to_int['X'] = aa_to_int['A'] # X -> A
if 'C' in aa_to_int: aa_to_int['U'] = aa_to_int['C'] # U -> C
if 'E' in aa_to_int: aa_to_int['Z'] = aa_to_int['E'] # Z -> E

## DATA PREPARATION

#### need to parse the txt motifs so that it is in list format (done)

In [207]:
import os

motif_dict = {}

for path in motif_paths:
    key = os.path.splitext(os.path.basename(path))[0]
    motif_dict[key] = []

    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line or set(line) == {"="}:
                continue
            motif_dict[key].append(line)


In [208]:
motif_dict

{'afgp_motifs': ['VTAAPAATAATAATPATAALNFAATAATPATPATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAAAAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAACNFAATAAT',
  'ATAAT',
  'IFAA',
  'AATAAT',
  'ATAA',
  'FAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATPAFNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALHFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAATPALIFAATAATAATPATPATPALIFAATAATAATPATPALNFAATAATAATTAARG'],
 'type1_motifs': ['AVTA', 'NAASAAA', 'AAAVAAATLEAAAAKAAATAVSAAAAAAAAAIAFAAAP'],
 'type2_motifs': ['LLVCAM', 'PVFR', 'LASIHS', 'WIGGS', 'VIPEVTPPSIM'],
 'type3_motifs'

#### need to grab access to each individual sequence so we can check if the motifs are within the sequence

In [209]:
df["type1_motifs"] = None
df["type2_motifs"] = None
df["type3_motifs"] = None
df["type4_motifs"] = None
df["afgp_motifs"] = None

In [210]:
df["type1_motifs"]

0       None
1       None
2       None
3       None
4       None
        ... 
4859    None
4860    None
4861    None
4862    None
4863    None
Name: type1_motifs, Length: 4864, dtype: object

In [211]:
def parse_fasta_to_dict(path: str):
   
    sequences = {}
    current_accession = None
    current_seq = []
    
    # Check if the file exists before attempting to open
    if not os.path.exists(path):
        print(f"Error: File not found at path: {path}")
        return {}

    try:
        with open(path, "r") as f:
            for line in f:
                line = line.strip()

                if not line:
                    continue

                # Header line starts with '>'
                if line.startswith(">"):
                    # Save the previous entry if one was being processed
                    if current_accession is not None:
                        # --- MODIFIED: Store only the sequence string ---
                        sequences[current_accession] = "".join(current_seq)

                    header_content = line[1:].strip() # Remove '>' and leading/trailing whitespace
                    current_seq = []
                    
                    # --- DUAL EXTRACTION LOGIC (Accession ID only) ---
                    
                    if "|" in header_content:
                        # 1. UniProt-Style (e.g., sp|H7FWB6|IBP_FLAFP ...)
                        parts = header_content.split("|")
                        if len(parts) >= 2:
                            # The Accession ID is between the first and second pipe
                            current_accession = parts[1]
                        else:
                            # Fallback for malformed pipe header
                            current_accession = header_content
                            
                    else:
                        # 2. Simple-Style (e.g., AAA74891.1 antifreeze protein ...)
                        first_space_index = header_content.find(" ")
                        
                        if first_space_index != -1:
                            current_accession = header_content[:first_space_index]
                        else:
                            # Handle case where the header is just ">Accession"
                            current_accession = header_content
                            
                else:
                    # Sequence line: append to the current sequence list
                    current_seq.append(line)

            # Save the last sequence after the loop finishes
            if current_accession is not None:
                # --- MODIFIED: Store only the sequence string ---
                sequences[current_accession] = "".join(current_seq)
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return {}

    return sequences

In [212]:
positive_reviewed_sequences = parse_fasta_to_dict(positive_reviewed_sequences_path)
positive_type1 = parse_fasta_to_dict(positive_type1_path)
positive_type2 = parse_fasta_to_dict(positive_type2_path)
positive_type3 = parse_fasta_to_dict(positive_type3_path)
positive_type4 = parse_fasta_to_dict(positive_type4_path)
merged_dict = positive_reviewed_sequences | positive_type1
merged_dict = merged_dict | positive_type2
merged_dict = merged_dict | positive_type3
positive_sequences = merged_dict | positive_type4
negative_sequences = parse_fasta_to_dict(negative_sequences_path)

master_sequences = positive_sequences | negative_sequences

In [213]:
positive_sequences

{'P04002': 'MALSLFTVGQLIFLFWTMRITEASPDPAAKAAPAAAAAPAAAAPDTASDAAAAAALTAANAKAAAELTAANAAAAAAATARG',
 'P24856': 'VTAAPAATAATAATPATAALNFAATAATPATPATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAAAAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAACNFAATAATPATAATPALIFAATAATAATPATAACNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATPAFNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALHFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATPAFHFAATAATPATAATPALIFAATAATAATPATAALNFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAALNFAATAATPATAATPALIFAATAATAATPATAAFNFAATAATAATPATAATPALIFAATAATAATPATPATPALIFAATAATAATPATPALNFAATAATAATTAARG',
 'B1P0S1': 'MALSLFTVGQFIFLFWTISITEANIDPAARAAAAAAASKAAVTAADAAAAAATIAASAASVAAATAADDAAASIATINAASAAAKS

In [214]:
negative_sequences

{'A0A0C5PHQ7': 'MEILNQRLNQQFDSWMGPRDPRVRGWLLLDNYLPTLSFTIIYLLIVWMGPKYMRNRQPVSCRGILVVYNMALTLLSLYMFYELVTAVWQGGYNFFCQDTHSGGEADNRVINVLWWYYFSKLIEFMDTFFFILRKNNHQITFLHIYHHFTMLNIWWFVMNWVPCGHSYFGATFNSFIHVLMYSYYGLSAIPAIQPYLWWKKYITQGQLVQFVLTMIQTSCAVVWPCGFPKGWLYFQISYMITLIILFSNFYIQTYKKKGTAAKKDPRHNGIKSVNGHSNGASHTNAVKNRKARTD',
 'A0A0G2KQY6': 'MTLRRASGCRQLTLTIGLALTLGLLQWPIGDVRGQDGASPAQVLQELLTRYGDNASISVPQLRSLLVRLNGGQSEDHDSKTQPTRTNASKCLAADTLAVYGMSEQSRIDERGLQQICPTMIQQLDSQACKTQPNQESESSPRPTEAEVWGYGLLCVTVISLCSLVGASVVPFMRKTFYKRLLLYFIALAIGTLYSNALFQLIPEAFGFDPMEDYYVPKSAVVFGGFYLFFFTEKILKMILKPKDTGGHGHGHSHFPAERYANSNGDLEDGVMEKLQNGEAGGAALPRAEADGRGVGEDDKMLSTGQTVQDTQSSGGGGTGGCYWLKGRAYSDIGTLAWMITLSDGLHNFIDGLAIGASFTASVFQGISTSVAILCEEFPHELGDFVILLNAGMSIQQALFFNFLSACCCYLGMGFGILAGNNFSPNWIFALAGGMFLYIALADMFPEMNEVSREEEEAGGSGFLLTFALQNAGLLTGFAIMLVLTIYSGQIQLG',
 'A0A0G2KTI4': 'MSASPPISAGDYLSAPEPDALKPAGPTPSQSRFQVDLVTESAGDGETTVGFDSSPPEYVAEPPPDGLRDSVSGGEEAKGRFRVVNFAASSPDAAPAETAQNGDTVMSEGSLHSSTGGQQHHHYDTHTNTYYLRTFGHNTIDAVPKIDFYRQTAAPLGE

In [215]:
# Itterate through all the rows in the df, 
# check the sequence_id column of the df 
# find the corresponding match in either negative_sequences or positive_sequences
# then check to find if any motif in motif_dict matches
# if there is a match, we want to fill these columns with either ones or zeros
# df["type1_motifs"] = None
# df["type2_motifs"] = None
# df["type3_motifs"] = None
# df["type4_motifs"] = None
# df["afgp_motifs"] = None
# so one row might have a match in both typ3 and afgp but no matches for other AFPs 
# It would look something like this for these columns:
# type1_motif type2_motif type3_motif type4_motif afgp_motif
# 0    0    1    0      1

# Map motif keys from motif_dict to df column names
motif_columns = ["type1_motifs", "type2_motifs", "type3_motifs", "type4_motifs", "afgp_motifs"]

# Ensure columns are initialized to 0
for col in motif_columns:
    df[col] = 0

# Iterate through DataFrame rows
for idx, row in df.iterrows():
    seq_id = row["sequence_id"]

    # Determine if seq_id is in positives or negatives
    if seq_id in positive_sequences:
        seq = positive_sequences[seq_id]
    elif seq_id in negative_sequences:
        seq = negative_sequences[seq_id]
    else:
        # No matching sequence → skip
        continue

    # Check motifs for every AFP type
    for motif_key in motif_columns:
        motifs = motif_dict[motif_key]

        # Check if any motif appears in the sequence
        has_match = any(m in seq for m in motifs)
        df.at[idx, motif_key] = 1 if has_match else 0


In [216]:
df = df[['sequence_id', 'label', 'reviewed_x', 'afp_type_x',"type1_motifs", "type2_motifs", "type3_motifs", "type4_motifs", "afgp_motifs"]]

#### Hydrophobicity and steric hinderance

In [217]:


from sequence_analysis.extract_features import steric_hindrance_score, hydrophobicity_score

# Create new columns
df["hydrophobicity"] = None
df["steric_hindrance"] = None

for idx, row in df.iterrows():
    seq_id = row["sequence_id"]

    # find sequence
    if seq_id in positive_sequences:
        seq = positive_sequences[seq_id]
    elif seq_id in negative_sequences:
        seq = negative_sequences[seq_id]
    else:
        # sequence not found
        continue

    # compute scores
    hydro_score = hydrophobicity_score(seq)
    steric_score = steric_hindrance_score(seq)

    # assign to dataframe
    df.at[idx, "hydrophobicity"] = hydro_score
    df.at[idx, "steric_hindrance"] = steric_score


#### Integrating non reviewed positive data (PBLAST) 

In [218]:
df["label"]

0           AFP
1           AFP
2           AFP
3           AFP
4           AFP
         ...   
4859    NON_AFP
4860    NON_AFP
4861    NON_AFP
4862    NON_AFP
4863    NON_AFP
Name: label, Length: 4864, dtype: object

In [219]:
# Create the new column 'detailed_label' for stratification purposes
def generate_detailed_label(row):
    # Condition 1: label is 'AFP' AND reviewed_x is 1
    if row['label'] == 'AFP' and row['reviewed_x'] == 1:
        return 'reviewed'
    # Condition 2: label is 'Non AFP' AND reviewed_x is 0
    elif row['label'] == 'NON_AFP' and row['reviewed_x'] == 1:
        return 'negative'
    # Condition 3: label is 'AFP' AND reviewed_x is 0
    elif row['label'] == 'AFP' and row['reviewed_x'] == 0:
        return 'non_reviewed'
    # Default case if none of the above are met (important for robustness)
    else:
        return 'other' # or None, or the original label

df['detailed_label'] = df.apply(generate_detailed_label, axis=1)

# Show a sample of the new column
print("--- DataFrame with New Column ---")
print(df[['label', 'detailed_label']].head())

# Show the distribution of the new column
print("\n--- Distribution of detailed_label ---")
print(df['detailed_label'].value_counts())


--- DataFrame with New Column ---
  label detailed_label
0   AFP       reviewed
1   AFP       reviewed
2   AFP       reviewed
3   AFP       reviewed
4   AFP       reviewed

--- Distribution of detailed_label ---
detailed_label
negative        4706
non_reviewed     132
reviewed          26
Name: count, dtype: int64


In [220]:
# MAKE NUMBER OF POSITIVE AND NEGATIVE SEQUENCES THE SAME
import pandas as pd

# Count how many AFP rows we have
num_afp = df[df["label"] == "AFP"].shape[0]

# Filter AFP and NON_AFP subsets
df_afp = df[df["label"] == "AFP"]
df_non_afp = df[df["label"] == "NON_AFP"]

# Randomly sample NON_AFP to match AFP count
df_non_afp_sampled = df_non_afp.sample(n=num_afp, random_state=42)

# Combine them into a balanced dataset
df_balanced = pd.concat([df_afp, df_non_afp_sampled], axis=0).sample(frac=1, random_state=42)

# Reset index for cleanliness
df_balanced = df_balanced.reset_index(drop=True)


In [221]:
df = df_balanced

#### ONE HOT ENCODING SOME FEATURES

In [222]:
df.select_dtypes(exclude=['float', 'int']).columns.tolist()

['sequence_id',
 'label',
 'afp_type_x',
 'hydrophobicity',
 'steric_hindrance',
 'detailed_label']

In [223]:
# 3. One-hot encode them

df.drop(["label"], inplace=True, axis=1)

In [224]:
df

,sequence_id,reviewed_x,afp_type_x,type1_motifs,type2_motifs,type3_motifs,type4_motifs,afgp_motifs,hydrophobicity,steric_hindrance,detailed_label
0,B9X0L5,1,NaN,0,0,0,0,0,0.012025,106.443038,negative
1,XP_068447920.1,0,type2,0,1,0,0,0,0.120833,100.75,non_reviewed
2,Q1KL12,1,NaN,0,0,0,0,0,-0.865947,98.561151,negative
3,XP_034721924.1,0,type2,0,1,0,0,0,-0.365306,102.122449,non_reviewed
4,AFQ32238.1,0,type3,0,0,1,0,0,0.421951,101.670732,non_reviewed
...,...,...,...,...,...,...,...,...,...,...,...
311,Q803Z2,1,NaN,0,0,0,0,0,0.263372,103.709302,negative
312,XP_044040401.1,0,type2,0,1,0,0,0,-0.387283,102.462428,non_reviewed
313,XP_019957605.1,0,type4,0,0,0,1,0,-0.280645,105.806452,non_reviewed
314,P81036,1,NaN,0,0,0,0,0,-0.232,87.733813,negative


In [225]:
df["detailed_label"].value_counts()

detailed_label
negative        158
non_reviewed    132
reviewed         26
Name: count, dtype: int64

## TRAINING THE MODEL

#### Splitting the dataset into 5 for each AFP type

In [226]:
from typing import Dict, Any
def split_for_binary_training(df: pd.DataFrame, target_col: str = 'afp_type_x') -> Dict[str, pd.DataFrame]:
    """
    Splits a DataFrame into multiple balanced DataFrames, one for each unique 
    non-NaN value in the target column. Each resulting DataFrame contains all 
    positive samples for that type and an equal number of randomly sampled 
    negative (NaN) samples.

    Args:
        df: The input pandas DataFrame.
        target_col: The name of the column containing the AFP types and NaN values.

    Returns:
        A dictionary where keys are the AFP types (e.g., 'Type_I') and values 
        are the corresponding balanced DataFrames.
    """
    if target_col not in df.columns:
        raise KeyError(f"Column '{target_col}' not found in DataFrame.")

    # 1. Separate the negative (NaN) samples
    # These are the rows where the AFP_type is NaN (the non-AFP group)
    df_negative_all = df[df[target_col].isna()].copy()

    # 2. Get the unique AFP types (positive classes)
    # We drop the NaN value to only iterate over the actual types
    positive_classes = df[target_col].dropna().unique()

    if len(positive_classes) == 0:
        print("Warning: No positive AFP types found in the data.")
        return {}

    # Dictionary to store the resulting balanced DataFrames
    balanced_dfs: Dict[str, pd.DataFrame] = {}

    print(f"Total negative (NaN) rows available for sampling: {len(df_negative_all)}")
    print(f"Identified positive classes: {list(positive_classes)}\n")

    # 3. Iterate through each AFP type to create a balanced subset
    for afp_type in positive_classes:
        # A. Filter for the positive samples of the current type
        df_positive = df[df[target_col] == afp_type].copy()
        
        # B. Determine the required sample size for balancing
        sample_size = len(df_positive)
        
        print(f"Processing {afp_type}: {sample_size} positive samples.")

        if sample_size == 0:
            print(f"Skipping {afp_type}: No positive samples found.")
            continue
            
        # C. Randomly sample the negative rows
        # We sample without replacement from the total negative pool.
        # Ensure that the sample size does not exceed the total available negative rows
        if sample_size > len(df_negative_all):
             print(f"Warning: Not enough negative rows ({len(df_negative_all)}) to balance {afp_type}. Using all available negative rows.")
             sample_size = len(df_negative_all)
             
        # Use .sample() to randomly select the required number of NaN rows
        df_negative_sample = df_negative_all.sample(n=sample_size, random_state=42, replace=False)
        
        # D. Combine the positive and the randomly sampled negative rows
        df_balanced = pd.concat([df_positive, df_negative_sample], ignore_index=True)
        
        print(f"   -> Resulting DataFrame for {afp_type} has {len(df_balanced)} rows ({sample_size} positive + {sample_size} negative).\n")
        
        # Store the result
        balanced_dfs[afp_type] = df_balanced

    return balanced_dfs

In [227]:
df_dict = split_for_binary_training(df)

Total negative (NaN) rows available for sampling: 158
Identified positive classes: ['type2', 'type3', 'afgp', 'type1', 'type4']

Processing type2: 57 positive samples.
   -> Resulting DataFrame for type2 has 114 rows (57 positive + 57 negative).

Processing type3: 24 positive samples.
   -> Resulting DataFrame for type3 has 48 rows (24 positive + 24 negative).

Processing afgp: 3 positive samples.
   -> Resulting DataFrame for afgp has 6 rows (3 positive + 3 negative).

Processing type1: 19 positive samples.
   -> Resulting DataFrame for type1 has 38 rows (19 positive + 19 negative).

Processing type4: 55 positive samples.
   -> Resulting DataFrame for type4 has 110 rows (55 positive + 55 negative).



In [228]:
df_dict.keys()

dict_keys(['type2', 'type3', 'afgp', 'type1', 'type4'])

In [229]:
df_dict["type2"]

,sequence_id,reviewed_x,afp_type_x,type1_motifs,type2_motifs,type3_motifs,type4_motifs,afgp_motifs,hydrophobicity,steric_hindrance,detailed_label
0,XP_068447920.1,0,type2,0,1,0,0,0,0.120833,100.75,non_reviewed
1,XP_034721924.1,0,type2,0,1,0,0,0,-0.365306,102.122449,non_reviewed
2,XP_044198199.1,0,type2,0,1,0,0,0,-0.702874,106.028736,non_reviewed
3,XP_046888434.1,0,type2,0,1,0,0,0,0.041497,102.85034,non_reviewed
4,XP_067101371.1,0,type2,0,1,0,0,0,0.171429,100.828571,non_reviewed
...,...,...,...,...,...,...,...,...,...,...,...
109,P79826,1,NaN,0,0,0,0,0,0.703488,105.224806,negative
110,Q90XN9,1,NaN,0,0,0,0,0,-0.858621,104.182759,negative
111,P22642,1,NaN,0,0,0,0,0,-0.381333,104.26,negative
112,Q6GMK6,1,NaN,0,0,0,0,0,0.41841,105.732218,negative


In [230]:
for afp in df_dict:
    df = df_dict[afp]
    df.drop(['reviewed_x', 'afp_type_x'], axis=1, inplace=True)

In [231]:
df = df_dict["type2"] # for now

#### CROSS VALIDATION INFO

In [390]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from hmmlearn.hmm import CategoricalHMM

# Helper function to convert amino acid sequences to numerical arrays
def sequence_to_array(sequence, aa_to_index):
    """Convert amino acid sequence string to numerical array"""
    return np.array([[aa_to_index.get(aa, 0)] for aa in sequence])

def prepare_hmm_data(sequence_ids, sequence_dict, aa_to_index):
    """
    Prepare sequences and lengths for HMM training from sequence IDs
    
    Parameters:
    -----------
    sequence_ids : array-like
        List of sequence IDs to include
    sequence_dict : dict
        Dictionary mapping sequence_id to sequence string
    aa_to_index : dict
        Dictionary mapping amino acid to index
        
    Returns:
    --------
    X : array, shape (n_samples, 1)
        Concatenated numerical sequences
    lengths : array
        Length of each sequence
    """
    sequences = []
    lengths = []
    
    for seq_id in sequence_ids:
        if seq_id in sequence_dict:
            seq_str = sequence_dict[seq_id]
            seq_array = sequence_to_array(seq_str, aa_to_index)
            sequences.append(seq_array)
            lengths.append(len(seq_array))
    
    if len(sequences) == 0:
        return np.array([]).reshape(0, 1), np.array([], dtype=np.int32)
    
    X = np.vstack(sequences)
    lengths = np.array(lengths, dtype=np.int32)
    
    return X, lengths

# Create stratification labels and binary target
stratify_labels = df['detailed_label'].values
df['Binary_Target'] = df['detailed_label'].apply(
    lambda x: 0 if x == 'negative' else 1
)
y = df['Binary_Target'].values

# Ensure sequence_id column exists in df
if 'sequence_id' not in df.columns:
    raise ValueError("DataFrame must contain 'sequence_id' column")

# Initialize StratifiedKFold
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# List to store evaluation scores
auc_scores = []
models = []

# Optimal number of components from your HMM hyperparameter tuning
n_components = 6

print(f"ready {n_splits}-fold Stratified Cross-Validation with HMM features...")


ready 5-fold Stratified Cross-Validation with HMM features...


#### TRAINING LOOP

In [391]:

# --- Cross-Validation Loop ---
for fold, (train_index, test_index) in enumerate(skf.split(df, stratify_labels)):
    print(f"\n{'='*70}")
    print(f"FOLD {fold+1}/{n_splits}")
    print(f"{'='*70}")
    
    # Split data for this fold
    df_train = df.iloc[train_index].copy()
    df_test = df.iloc[test_index].copy()
    y_train = y[train_index]
    y_test = y[test_index]
    
    # Get sequence IDs for training set
    train_sequence_ids = df_train['sequence_id'].values
    
    # Separate positive and negative sequence IDs in training set
    train_positive_ids = df_train[df_train['Binary_Target'] == 1]['sequence_id'].values
    train_negative_ids = df_train[df_train['Binary_Target'] == 0]['sequence_id'].values
    
    print(f"Training set: {len(train_positive_ids)} positive, {len(train_negative_ids)} negative sequences")
    
    # --- Train HMM Models ---
    print("\nTraining HMM models...")
    
    # Prepare positive HMM training data
    positive_X, positive_lengths = prepare_hmm_data(
        train_positive_ids, 
        positive_sequences, 
        aa_to_int
    )
    
    # Prepare negative HMM training data
    negative_X, negative_lengths = prepare_hmm_data(
        train_negative_ids, 
        negative_sequences, 
        aa_to_int
    )
    
    print(f"Positive HMM data: {positive_X.shape[0]} total samples, {len(positive_lengths)} sequences")
    print(f"Negative HMM data: {negative_X.shape[0]} total samples, {len(negative_lengths)} sequences")
    
    # Train positive model
    positive_model = CategoricalHMM(
        n_components=n_components,
        n_features=20,
        n_iter=100,
        random_state=42
    )
    positive_model.fit(positive_X, positive_lengths)
    
    # Train negative model
    negative_model = CategoricalHMM(
        n_components=n_components,
        n_features=20,
        n_iter=100,
        random_state=42)
    
    negative_model.fit(negative_X, negative_lengths)
    
    print("HMM models trained successfully!")
    
    # --- Generate HMM Features for Training Set ---
    print("\nGenerating HMM features for training set...")
    
    train_positive_log_probs = []
    train_negative_log_probs = []
    
    for seq_id in df_train['sequence_id']:
        # Try to find sequence in positive or negative dictionary
        if seq_id in positive_sequences:
            seq_str = positive_sequences[seq_id]
        elif seq_id in negative_sequences:
            seq_str = negative_sequences[seq_id]
        else:
            train_positive_log_probs.append(np.nan)
            train_negative_log_probs.append(np.nan)
            continue
        
        seq_array = sequence_to_array(seq_str, aa_to_int)
        
        # Score with both models
        pos_log_prob = positive_model.score(seq_array)
        neg_log_prob = negative_model.score(seq_array)
        
        train_positive_log_probs.append(pos_log_prob)
        train_negative_log_probs.append(neg_log_prob)
    
    # Add HMM features to training dataframe
    df_train['hmm_positive_log_prob'] = train_positive_log_probs
    df_train['hmm_negative_log_prob'] = train_negative_log_probs
    
    # --- Generate HMM Features for Test Set ---
    print("Generating HMM features for test set...")
    
    test_positive_log_probs = []
    test_negative_log_probs = []
    
    for seq_id in df_test['sequence_id']:
        if seq_id in positive_sequences:
            seq_str = positive_sequences[seq_id]
        elif seq_id in negative_sequences:
            seq_str = negative_sequences[seq_id]
        else:
            test_positive_log_probs.append(np.nan)
            test_negative_log_probs.append(np.nan)
            continue
        
        seq_array = sequence_to_array(seq_str, aa_to_int)
        
        pos_log_prob = positive_model.score(seq_array)
        neg_log_prob = negative_model.score(seq_array)
        
        test_positive_log_probs.append(pos_log_prob)
        test_negative_log_probs.append(neg_log_prob)
    
    # Add HMM features to test dataframe
    df_test['hmm_positive_log_prob'] = test_positive_log_probs
    df_test['hmm_negative_log_prob'] = test_negative_log_probs
    
    # --- Prepare Features for Logistic Regression ---
    # Drop columns that shouldn't be features
    columns_to_drop = ['detailed_label', 'Binary_Target', 'sequence_id']
    
    X_train = df_train.drop(columns=columns_to_drop)
    X_test = df_test.drop(columns=columns_to_drop)
    
    # Handle any NaN values (if sequences were missing)
    X_train = X_train.fillna(X_train.mean())
    X_test = X_test.fillna(X_train.mean())  # Use training mean for test set
    
    print(f"\nTraining features shape: {X_train.shape}")
    print(f"Test features shape: {X_test.shape}")
    print(f"Features: {list(X_train.columns)}")
    
    # --- Train Logistic Regression ---
    print("\nTraining Logistic Regression model...")
    
    model = LogisticRegression(solver='liblinear', random_state=42, max_iter=1000)
    model.fit(X_train, y_train)
    models.append(model)
    
    # Predict probabilities on the test set
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluate using AUC-ROC
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(roc_auc)
    
    print(f"\nFold {fold+1} Test AUC-ROC: {roc_auc:.4f}")
    
    # Show feature importances (coefficients)
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'coefficient': model.coef_[0]
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10).to_string(index=False))

# --- Final Results ---
print("\n" + "="*70)
print("FINAL RESULTS SUMMARY")
print("="*70)
print(f"Individual Fold AUC-ROC Scores: {[f'{score:.4f}' for score in auc_scores]}")
print(f"Mean AUC-ROC: {np.mean(auc_scores):.4f}")
print(f"Std Dev of AUC-ROC: {np.std(auc_scores):.4f}")

c:\Users\Alisha2\anaconda3\envs\bme_415\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  warnings.warn(



FOLD 1/5
Training set: 45 positive, 46 negative sequences

Training HMM models...
Positive HMM data: 7679 total samples, 45 sequences
Negative HMM data: 19090 total samples, 46 sequences
HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (91, 104)
Test features shape: (23, 104)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (91, 104)
Test features shape: (23, 104)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


...

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (91, 104)
Test features shape: (23, 104)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


HMM models trained successfully!

Generating HMM features for training set...
Generating HMM features for test set...

Training features shape: (92, 104)
Test features shape: (22, 104)
Features: ['length', 'net_charge_per_residue', 'hydrophobic_percent', 'polar_percent', 'charged_percent', 'gly_percent', 'pro_percent', 'ambiguous_count', 'ambiguous_rate', 'top1_identity', 'top1_score', 'top1_coverage', 'top2_identity', 'delta_top1_top2', 'type1_score', 'type1_aligned_len', 'type1_coverage', 'type1_identity', 'type1_mismatch_rate', 'type1_gap_rate', 'type1_conservation_match', 'type2_score', 'type2_aligned_len', 'type2_coverage', 'type2_identity', 'type2_mismatch_rate', 'type2_gap_rate', 'type2_conservation_match', 'type3_score', 'type3_aligned_len', 'type3_coverage', 'type3_identity', 'type3_mismatch_rate', 'type3_gap_rate', 'type3_conservation_match', 'type4_score', 'type4_aligned_len', 'type4_coverage', 'type4_identity', 'type4_mismatch_rate', 'type4_gap_rate', 'type4_conservation_ma

C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:128: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.fillna(X_train.mean())
C:\Users\Alisha2\AppData\Local\Temp\ipykernel_22144\398663222.py:129: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_test = X_test.fillna(X_train.mean())  # Use training mean for test set


## ORGANIZED PIPELINE 
### JUST PRESS PLAY UP TO THE SPLITTING THE DATASET SECTION (UNDER TRAINING THE MODEL SECTION) IN ORDER TO RUN 

In [204]:
df_dict["type2"]['detailed_label'].value_counts()

detailed_label
negative        57
non_reviewed    54
reviewed         3
Name: count, dtype: int64

In [232]:
import pandas as pd
import numpy as np
import json
import warnings
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, 
    recall_score, f1_score, matthews_corrcoef, confusion_matrix
)
from hmmlearn.hmm import CategoricalHMM
from Bio.Align import PairwiseAligner

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

class ProteinHMMPipeline:
    def __init__(self, sequence_dict, n_hmm_components=6, n_splits=5, random_state=42):
        """
        Args:
            sequence_dict (dict): Mapping of {sequence_id: "AMINO_ACID_STRING"}
            n_hmm_components (int): Hidden states for HMM
            n_splits (int): Number of CV folds
        """
        self.sequence_dict = sequence_dict
        self.n_components = n_hmm_components
        self.n_splits = n_splits
        self.random_state = random_state
        
        # Standard Amino Acid mapping
        self.aa_to_int = {aa: i for i, aa in enumerate("ACDEFGHIKLMNPQRSTVWY")}
        self.n_features = 20

        # Initialize Aligner once for speed
        self.aligner = PairwiseAligner()
        self.aligner.mode = 'global'
        self.aligner.match_score = 2
        self.aligner.mismatch_score = -1
        self.aligner.open_gap_score = -2
        self.aligner.extend_gap_score = -0.5

    # ---------------------------------------------------------
    # HMM Helpers
    # ---------------------------------------------------------
    def _sequence_to_array(self, sequence):
        return np.array([[self.aa_to_int.get(aa, 0)] for aa in sequence])

    def _prepare_hmm_data(self, sequence_ids):
        sequences = []
        lengths = []
        for seq_id in sequence_ids:
            if seq_id in self.sequence_dict:
                seq_str = self.sequence_dict[seq_id]
                seq_array = self._sequence_to_array(seq_str)
                sequences.append(seq_array)
                lengths.append(len(seq_array))
        
        if len(sequences) == 0:
            return np.array([]).reshape(0, 1), np.array([], dtype=np.int32)
        
        X = np.vstack(sequences)
        lengths = np.array(lengths, dtype=np.int32)
        return X, lengths

    # ---------------------------------------------------------
    # Medoid / Distance Helpers (Leak-Free Implementation)
    # ---------------------------------------------------------
    def _find_medoid_for_group(self, seq_ids):
        """
        Finds the 'Medoid' (central sequence) for a list of IDs.
        To keep this fast in a loop, we take a heuristic approach:
        The medoid is the sequence with the highest average pairwise alignment score 
        against a sample of the group.
        """
        # Get actual strings
        seqs = [self.sequence_dict[sid] for sid in seq_ids if sid in self.sequence_dict]
        if not seqs:
            return None, None

        # Optimization: If group is huge, sample 50 sequences to find the center
        sample_size = min(len(seqs), 50)
        candidates = seqs[:sample_size]
        
        best_score = -float('inf')
        best_seq = candidates[0]

        # Simple O(N) heuristic: The sequence closest to the median length is usually a good candidate
        # to start, but let's do real alignment for accuracy.
        for cand in candidates:
            current_score = 0
            for target in candidates:
                if cand == target: continue
                current_score += self.aligner.score(cand, target)
            
            if current_score > best_score:
                best_score = current_score
                best_seq = cand
        
        return best_seq

    def _calculate_alignment_features(self, seq, medoid_seq):
        """Calculates distance metrics between seq and a reference medoid."""
        if not medoid_seq:
            return {'score': 0, 'identity': 0, 'coverage': 0, 'gap_rate': 1}

        aln = self.aligner.align(seq, medoid_seq)[0]
        q_blocks, r_blocks = aln.aligned
        
        matches = 0
        seq_len = len(seq)
        med_len = len(medoid_seq)
        
        # Calculate matches manually (biopython aligner score includes gaps)
        # We need raw identity count
        matches = aln.counts().identities
        
        q_aligned = sum(qe - qs for qs, qe in q_blocks)
        aligned_len = max(1, q_aligned) # Avoid div/0
        
        identity = matches / aligned_len
        coverage = aligned_len / max(1, seq_len)
        
        # Gap calculation
        r_aligned = sum(re - rs for rs, re in r_blocks)
        gaps = (seq_len - q_aligned) + (med_len - r_aligned)
        gap_rate = gaps / max(1, (aligned_len + gaps))

        return {
            'score': aln.score,
            'identity': identity,
            'coverage': coverage,
            'gap_rate': gap_rate
        }
    def _generate_dynamic_features(self, target_ids, train_medoids):
        """
        Generates the 'Distance' and 'Medoid' features for a set of IDs
        based on the LEARNED medoids from the training set.
        """
        features_list = []
        
        for seq_id in target_ids:
            if seq_id not in self.sequence_dict:
                # Handle missing sequence case gracefully if needed, or skip
                continue
                
            seq = self.sequence_dict[seq_id]
            row_feats = {'sequence_id': seq_id}
            
            # 1. Basic Bio Params
            L = max(1, len(seq))
            row_feats['length'] = L
            row_feats['hydrophobic_percent'] = sum(seq.count(aa) for aa in 'AILMFPWYV') / L
            row_feats['charged_percent'] = sum(seq.count(aa) for aa in 'DEKR') / L
            row_feats['pro_percent'] = seq.count('P') / L
            
            # 2. Distance to Medoids (The "Top 1" Logic)
            type_scores = {}
            
            for type_name, medoid_seq in train_medoids.items():
                stats = self._calculate_alignment_features(seq, medoid_seq)
                
                # Store raw type features
                row_feats[f'{type_name}_score'] = stats['score']
                row_feats[f'{type_name}_identity'] = stats['identity']
                row_feats[f'{type_name}_coverage'] = stats['coverage']
                
                # Keep track for sorting
                type_scores[type_name] = stats

            # 3. Determine "Top 1" and "Top 2"
            # Sort by Identity (as per your request)
            sorted_types = sorted(type_scores.items(), key=lambda x: x[1]['identity'], reverse=True)
            
            if len(sorted_types) >= 1:
                top1_name, top1_data = sorted_types[0]
                row_feats['top1_score'] = top1_data['score']
                row_feats['top1_identity'] = top1_data['identity']
                row_feats['top1_type'] = top1_name # String, might need one-hot encoding if used as feature
                
                if len(sorted_types) >= 2:
                    top2_name, top2_data = sorted_types[1]
                    row_feats['delta_top1_top2'] = top1_data['identity'] - top2_data['identity']
                    row_feats['medoid_delta_top1_top2_score'] = top1_data['score'] - top2_data['score']
                else:
                    row_feats['delta_top1_top2'] = 0
                    row_feats['medoid_delta_top1_top2_score'] = 0
            else:
                row_feats['top1_score'] = 0
                row_feats['delta_top1_top2'] = 0
                row_feats['medoid_delta_top1_top2_score'] = 0

            features_list.append(row_feats)
            
        return pd.DataFrame(features_list)

    # ---------------------------------------------------------
    # Main Processing
    # ---------------------------------------------------------
    def process_single_df(self, df_name, df):
        print(f"\nProcessing Dataset: {df_name}")
        
        # Validation
        req_cols = ['detailed_label', 'sequence_id']
        if not all(col in df.columns for col in req_cols):
            raise ValueError(f"DF {df_name} missing columns. Required: {req_cols}")

        df = df.copy()
        # Create Binary Target
        df['Binary_Target'] = df['detailed_label'].apply(lambda x: 0 if x == 'negative' else 1)
        
        # We need 'detailed_label' for stratification and medoid building
        y = df['Binary_Target'].values
        detailed_labels = df['detailed_label'].values
        ids = df['sequence_id'].values
        
        # Identify "Existing Features" (columns passed in that aren't metadata)
        # We assume any column that ISN'T these reserved ones is a feature you want to keep
        reserved_cols = ['detailed_label', 'Binary_Target', 'sequence_id']
        existing_feature_cols = [c for c in df.columns if c not in reserved_cols]

        skf = StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=self.random_state)
        
        fold_results = []
        feature_importance_agg = []

        for fold, (train_idx, test_idx) in enumerate(skf.split(df, detailed_labels)):
            print(f"  > Running Fold {fold+1}/{self.n_splits}...")
            
            # --- 1. SPLIT DATA ---
            ids_train, ids_test = ids[train_idx], ids[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]
            labels_train = detailed_labels[train_idx]
            
            # Get existing features for this split
            X_train_existing = df.iloc[train_idx][existing_feature_cols].reset_index(drop=True)
            X_test_existing = df.iloc[test_idx][existing_feature_cols].reset_index(drop=True)
            
            # --- 2. TRAIN: LEARN THE MEDOIDS (Dynamic Profile Generation) ---
            # We only look at TRAIN data to find the "center" of each cluster
            unique_types = set(labels_train) - {'negative'}
            train_medoids = {}
            
            for type_label in unique_types:
                # Get IDs of this type in the training set
                type_ids = ids_train[labels_train == type_label]
                if len(type_ids) > 0:
                    medoid_seq = self._find_medoid_for_group(type_ids)
                    train_medoids[type_label] = medoid_seq
            
            # --- 3. TRANSFORM: GENERATE DYNAMIC FEATURES ---
            # Generate features for Train and Test using ONLY Train medoids
            X_train_dist = self._generate_dynamic_features(ids_train, train_medoids)
            X_test_dist = self._generate_dynamic_features(ids_test, train_medoids)
            
            # Identify columns that are numeric in the NEW features (drop ID, drop string types)
            numeric_cols = X_train_dist.select_dtypes(include=[np.number]).columns
            
            # Extract numeric parts and reset index to ensure concatenation alignment
            X_train_new_numeric = X_train_dist[numeric_cols].fillna(0).reset_index(drop=True)
            X_test_new_numeric = X_test_dist[numeric_cols].fillna(0).reset_index(drop=True)

            # --- 4. TRAIN: HMM ---
            # (Standard HMM logic as before)
            train_pos_ids = ids_train[y_train == 1]
            train_neg_ids = ids_train[y_train == 0]
            
            pos_X, pos_len = self._prepare_hmm_data(train_pos_ids)
            pos_hmm = CategoricalHMM(n_components=self.n_components, n_features=self.n_features, 
                                     n_iter=50, random_state=self.random_state, verbose=False)
            if len(pos_len) > 0: pos_hmm.fit(pos_X, pos_len)

            neg_X, neg_len = self._prepare_hmm_data(train_neg_ids)
            neg_hmm = CategoricalHMM(n_components=self.n_components, n_features=self.n_features, 
                                     n_iter=50, random_state=self.random_state, verbose=False)
            if len(neg_len) > 0: neg_hmm.fit(neg_X, neg_len)

            # Generate HMM scores
            def get_hmm_feats(seq_ids):
                p_scores, n_scores = [], []
                for seq_id in seq_ids:
                    if seq_id not in self.sequence_dict:
                        p_scores.append(-999); n_scores.append(-999)
                        continue
                    arr = self._sequence_to_array(self.sequence_dict[seq_id])
                    try: p_scores.append(pos_hmm.score(arr))
                    except: p_scores.append(-99999)
                    try: n_scores.append(neg_hmm.score(arr))
                    except: n_scores.append(-99999)
                return p_scores, n_scores

            tr_p, tr_n = get_hmm_feats(ids_train)
            te_p, te_n = get_hmm_feats(ids_test)
            
            # --- 5. COMBINE ALL FEATURES (Existing + New Medoid + New HMM) ---
            # Concatenate existing DF cols with new calculated cols
            X_train_final = pd.concat([X_train_existing, X_train_new_numeric], axis=1)
            X_train_final['hmm_pos'] = tr_p
            X_train_final['hmm_neg'] = tr_n
            
            X_test_final = pd.concat([X_test_existing, X_test_new_numeric], axis=1)
            X_test_final['hmm_pos'] = te_p
            X_test_final['hmm_neg'] = te_n

            # Handle NaN in existing features if any
            X_train_final = X_train_final.fillna(0)
            X_test_final = X_test_final.fillna(0)
            
            # --- 6. PREDICT (Logistic Regression) ---
            # Check for object type columns that might sneak in (like 'top1_type' string)
            # and drop them for the model
            X_train_final = X_train_final.select_dtypes(include=[np.number])
            X_test_final = X_test_final.select_dtypes(include=[np.number])

            lr = LogisticRegression(solver='liblinear', max_iter=1000, random_state=self.random_state)
            lr.fit(X_train_final, y_train)
            
            y_pred_proba = lr.predict_proba(X_test_final)[:, 1]
            y_pred = lr.predict(X_test_final)

            # Metrics
            metrics = self._calculate_detailed_metrics(y_test, y_pred, y_pred_proba)
            metrics['fold'] = fold + 1
            fold_results.append(metrics)
            
            feature_importance_agg.append(pd.Series(lr.coef_[0], index=X_train_final.columns))

        # Aggregate
        metrics_df = pd.DataFrame(fold_results)
        agg_metrics = metrics_df.mean(numeric_only=True).to_dict()
        std_metrics = metrics_df.std(numeric_only=True).to_dict()
        avg_features = pd.concat(feature_importance_agg, axis=1).mean(axis=1).sort_values(key=abs, ascending=False)

        return {
            "summary": {
                "mean_metrics": agg_metrics,
                "std_dev_metrics": std_metrics
            },
            "feature_importance": avg_features.to_dict(),
            "fold_details": fold_results
        }

    def _calculate_detailed_metrics(self, y_true, y_pred, y_proba):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        return {
            "auc_roc": float(roc_auc_score(y_true, y_proba)),
            "accuracy": float(accuracy_score(y_true, y_pred)),
            "precision": float(precision_score(y_true, y_pred, zero_division=0)),
            "recall_sensitivity": float(recall_score(y_true, y_pred, zero_division=0)),
            "specificity": float(specificity),
            "f1_score": float(f1_score(y_true, y_pred, zero_division=0)),
            "mcc": float(matthews_corrcoef(y_true, y_pred)),
            "confusion_matrix": {"tp": int(tp), "tn": int(tn), "fp": int(fp), "fn": int(fn)}
        }

    def run_batch(self, df_dict, output_json_path="results_no_leakage.json"):
        final_output = {}
        for name, df in df_dict.items():
            try:
                final_output[name] = self.process_single_df(name, df)
            except Exception as e:
                print(f"Error processing {name}: {str(e)}")
                final_output[name] = {"error": str(e)}
        
        with open(output_json_path, 'w') as f:
            json.dump(final_output, f, indent=4)
        print(f"\nPipeline complete. Results saved to {output_json_path}")
        return final_output

# ==========================================
# EXAMPLE USAGE
# ==========================================

# 1. Define your master sequence dictionary
# master_seq_dict = {
#     "seq1": "MKV...", 
#     "seq2": "ALI...",
#     ...
# }

# 2. Load your DataFrames into a dictionary
# dfs = {
#     "AFP_Type_1": df_type1,
#     "AFP_Type_2": df_type2
# }

# 3. Run Pipeline
pipeline = ProteinHMMPipeline(sequence_dict=master_sequences, n_hmm_components=6)
results = pipeline.run_batch(df_dict, output_json_path="..\..\model_results\protein_analysis_metrics.json")


Processing Dataset: type2
  > Running Fold 1/5...
  > Running Fold 2/5...
  > Running Fold 3/5...
  > Running Fold 4/5...
  > Running Fold 5/5...

Processing Dataset: type3
  > Running Fold 1/5...
  > Running Fold 2/5...
  > Running Fold 3/5...
  > Running Fold 4/5...
  > Running Fold 5/5...

Processing Dataset: afgp
Error processing afgp: n_splits=5 cannot be greater than the number of members in each class.

Processing Dataset: type1
  > Running Fold 1/5...
  > Running Fold 2/5...
  > Running Fold 3/5...
  > Running Fold 4/5...
  > Running Fold 5/5...

Processing Dataset: type4
  > Running Fold 1/5...
  > Running Fold 2/5...
  > Running Fold 3/5...
  > Running Fold 4/5...
  > Running Fold 5/5...

Pipeline complete. Results saved to ..\..\model_results\protein_analysis_metrics.json
